# 自己紹介

<table style="width:image width px; font-size:100%; text-align:left;">
    <tr>
        <td><img src="img/自己紹介.JPG" width="500"></td>
        <td>&nbsp;</td>
        <td>【名前】 こっしー <br>
    【職業】 新卒機械学習エンジニア/データサイエンティスト <br>
    【興味関心】 NLP(文書要約、平易化)、因果推論、ベイズ機械学習 <br>
    【趣味】 ミュージカル、街歩き</td>
    </tr>
</table>

**第6章**

- 導入
    - API とは？
    - Web アプリケーションとは？
    - API、Web application はいつ使うと嬉しいのか
    - 本章で扱う問題紹介
- 数理モデリングと実装
- 最適化 API を作る
- Web アプリケーションを作る

こういう勉強会に不慣れなので、基本的に自分の勉強の記録を公開するという形式.  
Web アプリケーションは時間が足りないと思うので、端折ってある。  
仕事の方が忙しく、4 章の一部と 5 章の発表を全然聞けなかったのが残念です.   

# 導入

## API とは？
<div style="width:image width px; font-size:100%; text-align:center;"><img src='img/API.png' alt="alternate text" width="width" height="height" style="width:1300px;height:400px;" /> API 説明図 </div>

API: Application Programming Interface. 本文ママ「汎用性の高い機能を、誰でも手軽に利用できる様に提供される便利なしくみ」  
リクエストとレスポンスのこの仕組みが API.

## Web Application とは
<div style="width:image width px; font-size:100%; text-align:center;"><img src='img/WebApplication.png' alt="alternate text" width="width" height="height" style="width:1300px;height:400px;" /> Web Application 説明図 </div>

リクエストとレスポンスという構造自体はAPI と似ているが、Web 上でボタンを押したり、ファイルをアップロードしたりを Web ブラウザ上で行うという点が異なる。  

一般に、インターネットなどのネットワークを経由して利用するアプリケーションソフトウェア。
web サーバー上で動く.  

今回のように、API を用いて Web application を動かしたい場合は、  
例えばファイル/データをアップロードして、それを何とかしてくれというリクエストを送り、レスポンスとして分析結果が返ってきて、それをリクエスト送信者のweb上で表示するとする。  
~~upload data, display=web application. uploaded data into request, response=API という認識。~~

## なぜ API と Web application を学ぶのか
- ビジネス観点のメリット
    - 短期間で PoC を自力推進できる
    - プロマネ / 顧客 / 上司にイメージを持ってもらいやすい
- プロダクト観点のメリット
    - エンジニアとのコミュニケーションが円滑に
    - プロダクトを俯瞰できる

PoC = Proof of Cpncept. 実際に機能として思い浮かべた概念を実装できるのか確かめる行為.  
Product. merit.: エンジニアとのcommunication = 作業が効率化。何を決める必要があるかエンジニアにイメージを持ってもらいやすい.  
プロダクトを俯瞰: プロダクト側から見て嬉しい実装方法などが分かる。問題の早期発見につながる.  

Web application 実装に、今回は Flask というフレームワークを使う。  
他にも色々フレームワークはある。最近だと、streamlit を個人的に良く聞く気がします。  
今回は恐らく時間が足りないので、streamlit は触れません。  
--> 金曜日発表の方の資料がめちゃくちゃ面白いので見てみると良いかと思います！  

## API、Web Application があると嬉しいケース

営業「どの順番で営業訪問すればいいか教えて。費用は最小限で。」  
わたし「訪問先リストくれたら算出するよ」  

翌日  
営業「訪問先リスト変わった」  
わたし「もう一回算出するわ」  

数日後  
営業「訪問先リスト変わった」  
営業「他の営業の分も最適化して」  
わたし「過労死」  

-> Web application があれば、営業に自分でルート算出してもらえた。  
Web application って役に立ちそう

## 本章で扱う問題 
あなたは大学生。サークルのメンバーで、車を使った旅行をする。誰をどの車に乗せるべき？

ここからは実際に web アプリケーションを作って行きたい。

# 数理モデリングと実装

数理最適化問題にすると、**0-1整数計画問題**

- 概要：学生のグループ分け問題
    - 要件(1) 学生をどの車に割り当てるかを決定する
- 法規制による制約
    - 要件(2) 乗車人数は定員を超えてはいけない
    - 要件(3) 運転免許証を持つ学生を 1 人以上車に割り当てる

0-1 整数計画 = 学生と車 id の組という変数 ->  0/1 

- 親睦を深める制約
    - 要件(4) 各学年の学生を 1 人以上車に割り当てる
- ジェンダーバランスによる制約
    - 要件(5) 男女それぞれ 1 人以上を車に割り当てる

## データを概観したい

In [1]:
import pandas as pd

students_df = pd.read_csv('../PyOptBook-main/6.api/resource/students.csv')
print(students_df.shape)

(24, 4)


In [2]:
students_df.head(5)

,student_id,license,gender,grade
0,0,0,0,1
1,1,1,1,2
2,2,1,0,3
3,3,1,1,4
4,4,0,1,1


In [3]:
students_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   student_id  24 non-null     int64
 1   license     24 non-null     int64
 2   gender      24 non-null     int64
 3   grade       24 non-null     int64
dtypes: int64(4)
memory usage: 896.0 bytes


In [4]:
cars_df = pd.read_csv('../PyOptBook-main/6.api/resource/cars.csv')
print(cars_df.shape)

(6, 2)


In [5]:
display(cars_df)

,car_id,capacity
0,0,6
1,1,6
2,2,5
3,3,4
4,4,5
5,5,5


## 数理モデリング

- リスト
    - $S$: 学生のリスト
    - $C$: 車のリスト
    - $G$: 学年のリスト
    - $S_{license}$: 免許を持っている学生のリスト
    - $S_g  (g\in{G})$: 学年が $g$ の学生のリスト
    - $S_{male}$: 男性のリスト
    - $S_{female}$: 女性のリスト

- 変数: 学生 $s$ を車 $c$ に割り当てる場合に 1、そうでない場合に 0 をとる変数 
    - $x_{sc}\in\{0,1\}$
    
- 定数
    - $U_c$ ($c\in{C}$): 車 $c$ の乗車定員

- 制約条件
    - 制約 (1) 各学生を 1 つの車に割り当てる
        - $\sum_{c\in{C}}x_{sc} = 1 \quad (s\in S)$
    - 制約 (2) 各車には乗車定員より多く乗ることができない
        - $\sum_{s\in S}x_{sc}\leq U_c \quad (c\in C)$
    - 制約 (3) 各車にドライバーを 1 人以上割り当てる
        - $\sum_{s\in {S_{license}}}x_{sc}\geq 1 \quad (c\in C)$

- 制約条件
    - 制約 (4) 各車に各学年の学生を 1 人以上割り当てる
        - $\sum_{s\in {S_g}}x_{sc}\geq 1 \quad (g \in{G}, c\in{C})$
    - 制約 (5) 各車に男性を 1 人以上割り当てる
        - $\sum_{s\in {S_{male}}}x_{sc}\geq 1 \quad (c\in{C})$
    - 制約 (6) 各車に女性を 1 人以上割り当てる
        - $\sum_{s\in {S_{female}}}x_{sc}\geq 1 \quad (c\in{C})$
- 目的関数
    - なし

## 数理モデルの実装

In [25]:
import pandas as pd
import pulp

students_df = pd.read_csv('../PyOptBook-main/6.api/resource/students.csv')
cars_df = pd.read_csv('../PyOptBook-main/6.api/resource/cars.csv')

In [26]:
# 学生の乗車グループ分け問題 (0-1 整数計画問題) のインスタンス作成
prob = pulp.LpProblem('ClubCarProblem', pulp.LpMaximize)

In [27]:
# リスト
## 学生のリスト
S = students_df['student_id'].to_list()
## 車のリスト
C = cars_df['car_id'].to_list()
## 学年のリスト
G = [1, 2, 3, 4]
## 学生と車のペアのリスト
SC = [(s,c) for s in S for c in C]

SC の計算にかかる時間を確認

In [28]:
%%timeit
SC = [(s,c) for s in S for c in C]

8.93 µs ± 510 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [29]:
## 免許を持っている学生のリスト
S_license = students_df[students_df['license']==1]['student_id']
S_license2 = students_df.loc[students_df['license']==1, 'student_id']
(S_license == S_license2).all()

True

S_license の計算にかかる時間を確認

In [30]:
%%timeit
S_license = students_df[students_df['license']==1]['student_id']

286 µs ± 18.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
%%timeit
S_license2 = students_df.loc[students_df['license']==1, 'student_id']

207 µs ± 39.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [32]:
## 学年が g の学生のリスト
S_g0 = {g: students_df.loc[students_df['grade']==g, 'student_id'] 
       for g in G}
S_g = {g: students_df[students_df['grade']==g]['student_id'] 
       for g in G}

In [33]:
print(S_g0.keys() == S_g.keys())
print(S_g0.values() == S_g.values())

print([(S_g0[key] == S_g[key]).all() for key in S_g.keys()])


True
False
[True, True, True, True]


In [34]:
print(S_g0.values()) 
print('---------------------')
print(S_g.values())

dict_values([0      0
4      4
8      8
12    12
16    16
20    20
Name: student_id, dtype: int64, 1      1
5      5
9      9
13    13
17    17
21    21
Name: student_id, dtype: int64, 2      2
6      6
10    10
14    14
18    18
22    22
Name: student_id, dtype: int64, 3      3
7      7
11    11
15    15
19    19
23    23
Name: student_id, dtype: int64])
---------------------
dict_values([0      0
4      4
8      8
12    12
16    16
20    20
Name: student_id, dtype: int64, 1      1
5      5
9      9
13    13
17    17
21    21
Name: student_id, dtype: int64, 2      2
6      6
10    10
14    14
18    18
22    22
Name: student_id, dtype: int64, 3      3
7      7
11    11
15    15
19    19
23    23
Name: student_id, dtype: int64])


S_g の計算にかかる時間を確認

In [35]:
%%timeit
S_g0 = {g: students_df.loc[students_df['grade']==g, 'student_id'] 
       for g in G}

780 µs ± 47.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [36]:
%%timeit
S_g = {g: students_df[students_df['grade']==g]['student_id'] 
       for g in G}

1.06 ms ± 23.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [37]:
## 男性と女性のリスト
S_male = students_df[students_df['gender']==0]['student_id']
S_female = students_df[students_df['gender']==1]['student_id']

In [38]:
S_male = students_df[students_df['gender']==0]['student_id']
S_male0 = students_df.loc[students_df['gender']==0, 'student_id']
(S_male0 == S_male).all()

True

S_male の計算にかかる時間を確認

In [39]:
%%timeit
S_male0 = students_df.loc[students_df['gender']==0, 'student_id']

207 µs ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [40]:
%%timeit
S_male = students_df[students_df['gender']==0]['student_id']

302 µs ± 48.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [41]:
# 定数
## 車の乗車定員の定数
U = cars_df['capacity'].to_list()

In [42]:
# 変数
## 学生をどの車に割り当てるかを変数として定義
x = pulp.LpVariable.dicts('x', SC, cat='Binary')

In [43]:
# 制約
## (1) 各学生を １ つの車に割り当てる
for s in S:
    prob += pulp.lpSum([x[s,c] for c in C]) == 1
    
## (2) 法規制に関する制約: 各車には乗車定員より多く乗ることができない
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S]) <= U[c]

In [44]:
## (3) 法規制に関する制約： 各車にドライバーを1人以上割り当てる
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_license]) >= 1
    
## (4) 懇親を目的とした制約: 各車に各学年の学生を1人以上割り当てる
for c in C:
    for g in G:
        prob += pulp.lpSum([x[s,c] for s in S_g[g]]) >= 1

In [45]:
## (5) ジェンダーバランスを考慮した制約: 各車に男性を1人以上割り当てる
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_male]) >= 1

## (5) ジェンダーバランスを考慮した制約: 各車に女性を1人以上割り当てる
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_female]) >= 1

## 問題を解く

In [46]:
# 求解
solver = pulp.SCIP()
status = prob.solve(solver)

print('Status: ', pulp.LpStatus[status])

Status:  Optimal


In [47]:
# 最適化結果の表示
# 各車に割り当てられている学生のリストを辞書に格納（車 ID -> 割り当てられた学生のリスト）
car2students = {
    c: [s for s in S if x[s,c].value()==1] 
    for c in C
}

# 各車の乗車定員（車 ID -> 乗車定員）
max_people = dict(
    zip(cars_df['car_id'], cars_df['capacity'])
)

In [48]:
for c, ss in car2students.items():
    print(f'車 ID: {c}')
    print(f'学生数（乗車定員）: {len(ss)}({max_people[c]})')
    print(f'学生: {ss}\n')

車 ID: 0
学生数（乗車定員）: 4(6)
学生: [2, 5, 7, 8]

車 ID: 1
学生数（乗車定員）: 4(6)
学生: [12, 15, 21, 22]

車 ID: 2
学生数（乗車定員）: 4(5)
学生: [17, 18, 20, 23]

車 ID: 3
学生数（乗車定員）: 4(4)
学生: [10, 13, 16, 19]

車 ID: 4
学生数（乗車定員）: 4(5)
学生: [0, 1, 3, 14]

車 ID: 5
学生数（乗車定員）: 4(5)
学生: [4, 6, 9, 11]



## 確認

In [52]:
# 解を、車と学生の df と結合できる形式にする
car2students_df = pd.DataFrame(car2students).T\
    .stack()\
    .reset_index(drop=False)
car2students_df = car2students_df.drop(car2students_df.columns[1], axis=1)
car2students_df.columns = ['car_id', 'student_id']

# 結合する
validation_df = pd.merge(
    car2students_df, students_df, how='left'
)
validation_df = pd.merge(
    validation_df, cars_df, how='left'
)

今回の制約を確認するだけなら、この df を作る必要はなかったかも...

In [53]:
validation_df

,car_id,student_id,license,gender,grade,capacity
0,0,2,1,0,3,6
1,0,5,0,1,2,6
2,0,7,0,1,4,6
3,0,8,1,1,1,6
4,1,12,1,1,1,6
5,1,15,1,0,4,6
6,1,21,0,0,2,6
7,1,22,1,1,3,6
8,2,17,1,0,2,5
9,2,18,0,1,3,5


In [54]:
# 学生と車がきちんと全員、全車、解の中に存在するか調べる
print(set(S) == set(validation_df['student_id']))
print(set(C) == set(validation_df['car_id']))

True
True


In [55]:
# 制約(1) 各学生が一つの車に割り当てられているか
validation_df['student_id'].value_counts()

2     1
5     1
9     1
6     1
4     1
14    1
3     1
1     1
0     1
19    1
16    1
13    1
10    1
23    1
20    1
18    1
17    1
22    1
21    1
15    1
12    1
8     1
7     1
11    1
Name: student_id, dtype: int64

In [33]:
# 制約(2) 各車が乗車定員オーバーしていないか
for c in validation_df['car_id'].unique():
    tmp_df = validation_df[validation_df['car_id']==c].copy()
    display(tmp_df)
    print(all(len(tmp_df) <= tmp_df['capacity']))

,car_id,student_id,license,gender,grade,capacity
0,0,2,1,0,3,6
1,0,5,0,1,2,6
2,0,7,0,1,4,6
3,0,8,1,1,1,6


True


,car_id,student_id,license,gender,grade,capacity
4,1,12,1,1,1,6
5,1,15,1,0,4,6
6,1,21,0,0,2,6
7,1,22,1,1,3,6


True


,car_id,student_id,license,gender,grade,capacity
8,2,17,1,0,2,5
9,2,18,0,1,3,5
10,2,20,1,1,1,5
11,2,23,1,1,4,5


...

True


,car_id,student_id,license,gender,grade,capacity
16,4,0,0,0,1,5
17,4,1,1,1,2,5
18,4,3,1,1,4,5
19,4,14,1,0,3,5


True


,car_id,student_id,license,gender,grade,capacity
20,5,4,0,1,1,5
21,5,6,0,0,3,5
22,5,9,1,0,2,5
23,5,11,0,1,4,5


True


In [59]:
# 制約(2) 別検証
for i, k in enumerate(car2students.keys()):
    print(len(car2students[k]) <= U[i])

True
True
True
True
True
True


In [34]:
# 制約(3) 各車にドライバーが1人以上いるか
validation_df.groupby('car_id')['license'].sum()

car_id
0    2
1    3
2    3
3    4
4    3
5    1
Name: license, dtype: int64

In [35]:
# 制約(3) 各車にドライバーが1人以上いるか
for c in validation_df['car_id'].unique():
    tmp_df = validation_df[validation_df['car_id']==c].copy()
    display(tmp_df)
    print(sum(tmp_df['license']) >= 1)
    
#### -> 所感: car_id 5 はどうにかした方がいいかもしれない。ドライバーが1人しかいない

,car_id,student_id,license,gender,grade,capacity
0,0,2,1,0,3,6
1,0,5,0,1,2,6
2,0,7,0,1,4,6
3,0,8,1,1,1,6


True


,car_id,student_id,license,gender,grade,capacity
4,1,12,1,1,1,6
5,1,15,1,0,4,6
6,1,21,0,0,2,6
7,1,22,1,1,3,6


True


,car_id,student_id,license,gender,grade,capacity
8,2,17,1,0,2,5
9,2,18,0,1,3,5
10,2,20,1,1,1,5
11,2,23,1,1,4,5


...

True


,car_id,student_id,license,gender,grade,capacity
16,4,0,0,0,1,5
17,4,1,1,1,2,5
18,4,3,1,1,4,5
19,4,14,1,0,3,5


True


,car_id,student_id,license,gender,grade,capacity
20,5,4,0,1,1,5
21,5,6,0,0,3,5
22,5,9,1,0,2,5
23,5,11,0,1,4,5


True


In [36]:
# 制約(4) 各車に各学年の学生を1人以上割り当てているか
validation_df.groupby('car_id')['grade'].count() == 4

car_id
0    True
1    True
2    True
3    True
4    True
5    True
Name: grade, dtype: bool

In [37]:
# 制約(4) 各車に各学年の学生を1人以上割り当てているか
for c in validation_df['car_id'].unique():
    tmp_df = validation_df[validation_df['car_id']==c].copy()
    print(len(tmp_df['grade'].unique()) == 4)

True
True
True
True
True
True


In [38]:
# 制約（５）, (6) 男女1人ずつ以上いるか
validation_df.groupby('car_id')['gender'].mean()

car_id
0    0.75
1    0.50
2    0.75
3    0.50
4    0.50
5    0.50
Name: gender, dtype: float64

In [39]:
# 制約（５）, (6) 男女1人ずつ以上いるか
for c in validation_df['car_id'].unique():
    tmp_df = validation_df[validation_df['car_id']==c].copy()
    print(tmp_df['gender'].nunique() == 2)

True
True
True
True
True
True


# 最適化 API を作る

まずは要件定義  
==> 何を作りたいのか明確にする  
==> その後、その要件を達成するために API が備えるべき機能を考える（仕様）

【要件】  
API に学生データと車データを投げると、学生の乗車グループ分け問題を解いた結果を得られる。    
 
【仕様】  
・HTTP プロトコルによる API との通信   
リクエスト(入力)：学生データ、車データの csv ファイル  
レスポンス(出力)：最適化結果の csv ファイル  
・最適化(グループ分け)を行うモジュールの作成が必要  

上述のように、今回は Flask を使う。  
==> HTTP 形式での通信を行う仕組みをほぼ自動で作成してくれる。   

今回作る必要がある要素  
・リクエスト(入力)を受け付ける部分
・最適化を行う部分  
・レスポンス(出力)を返す部分  

完成イメージコード

```python
def solve():
    # 1. リクエスト受信
    students_df, cars_df = preprocess(request)
    # 2. 最適化実行
    solution_df = CarGroupProblem(students_df, cars_df).solve()
    # 3. レスポンス返送
    response = postprocess(solution_df)
    
    return response
```

<div style="width:image width px; font-size:100%; text-align:center;"><img src='img/API_case_study.png' alt="alternate text" width="width" height="height" style="width:1300px;height:400px;" /> 通信フローとファイルの関係性 </div>

apiの図。リクエストとしてcsvをもらい、それをサーバーに送り、サーバー側で最適化を実行。   
出力として返ってきた結果をcsvのレスポンスとして送り返す仕組み

まず最適化をするためのモジュールを作ろう。  
- __init__() : クラスを用いてインスタンスを立てる際にインスタンスを初期化するコード
- _formulate() ： 先頭に single underscore　があるメソッドは、非公開。　
    - ”アンダースコアで始まる名前 (例えば _spam) は、 (関数であれメソッドであれデータメンバであれ) 非 public なAPIとして扱います。これらは、予告なく変更されるかもしれない実装の詳細として扱われるべきです。”
    - [python のチュートリアル](https://docs.python.org/ja/3/tutorial/classes.html#private-variables)より引用

- if __name__ = '__main__' : モジュールとして import される場合でなく、py スクリプトファイルとして実行される場合のみ実行するという意味。  

In [40]:
import pandas as pd
import pulp


class CarGroupProblem():
    """学生の乗車グループ分け問題を解くクラス"""
    def __init__(self, students_df, cars_df, name='ClubCarProblem'):
        # 初期化メソッド
        self.students_df = students_df
        self.cars_df = cars_df
        self.name = name
        self.prob = self._formulate()

    def _formulate(self):
        # 学生の乗車グループ分け問題（0-1整数計画問題）のインスタンス作成
        prob = pulp.LpProblem(self.name, pulp.LpMinimize)

        # リスト
        # 学生のリスト
        S = self.students_df['student_id'].to_list()
        # 車のリスト
        C = self.cars_df['car_id'].to_list()
        # 学年のリスト
        G = [1, 2, 3, 4]
        # 学生と車のペアのリスト
        SC = [(s, c) for s in S for c in C]
        # 免許を持っている学生のリスト
        S_license = self.students_df[self.students_df['license'] == 1]['student_id']
        # 学年が  g  の学生のリスト
        S_g = {g: self.students_df[self.students_df['grade'] == g]['student_id'] for g in G}
        # 男性と女性のリスト
        S_male = self.students_df[self.students_df['gender'] == 0]['student_id']
        S_female = self.students_df[self.students_df['gender'] == 1]['student_id']

        # 定数
        # 車の乗車定員
        U = self.cars_df['capacity'].to_list()

        # 変数
        # 学生をどの車に割り当てるかを変数として定義
        x = pulp.LpVariable.dicts('x', SC, cat='Binary')

        # 制約
        # (1) 各学生を１つの車に割り当てる
        for s in S:
            prob += pulp.lpSum([x[s, c] for c in C]) == 1

        # (2) 法規制に関する制約：各車には乗車定員より多く乗ることができない
        for c in C:
            prob += pulp.lpSum([x[s, c] for s in S]) <= U[c]

        # (3) 法規制に関する制約：各車にドライバーを1人以上割り当てる
        for c in C:
            prob += pulp.lpSum([x[s, c] for s in S_license]) >= 1

        # (4) 懇親を目的とした制約: 各車に各学年の学生を１人以上割り当てる
        for c in C:
            for g in G:
                prob += pulp.lpSum([x[s, c] for s in S_g[g]]) >= 1

        # (5) 各車に男性を1人以上割り当てる
        for c in C:
            prob += pulp.lpSum([x[s, c] for s in S_male]) >= 1

        # (6) 各車に女性を1人以上割り当てる
        for c in C:
            prob += pulp.lpSum([x[s, c] for s in S_female]) >= 1

        # 最適化後に利用するデータを返却
        return {'prob': prob, 'variable': {'x': x}, 'list': {'S': S, 'C': C}}

    def solve(self):
        # 最適化問題を解くメソッド
        # 問題を解く
        solver = pulp.SCIP()
        status = self.prob['prob'].solve(solver)

        # 最適化結果を格納
        x = self.prob['variable']['x']
        S = self.prob['list']['S']
        C = self.prob['list']['C']
        car2students = {c: [s for s in S if x[s, c].value() == 1] for c in C}
        student2car = {s: c for c, ss in car2students.items() for s in ss}
        solution_df = pd.DataFrame(list(student2car.items()), columns=['student_id', 'car_id'])

        return solution_df


if __name__ == '__main__':
    # データの読み込み
    students_df = pd.read_csv('../PyOptBook-main/6.api/resource/students.csv')
    cars_df = pd.read_csv('../PyOptBook-main/6.api/resource/cars.csv')

    # 数理モデル インスタンスの作成
    prob = CarGroupProblem(students_df, cars_df)

    # 問題を解く
    solution_df = prob.solve()

    # 結果の表示
    print('Solution: \n', solution_df)


Solution: 
     student_id  car_id
0            2       0
1            5       0
2            7       0
3            8       0
4           12       1
5           15       1
6           21       1
7           22       1
8           17       2
9           18       2
10          20       2
11          23       2
12          10       3
13          13       3
14          16       3
15          19       3
16           0       4
17           1       4
18           3       4
19          14       4
20           4       5
21           6       5
22           9       5
23          11       5


## Flask を使った API 作成
実際に Flask で API を作ってみよう。まずはコードの全体像を確認する

```python
# 必要なライブラリのインポート
from flask import Flask, request
from problem import CarGroupProblem

# Flask のアプリケーションを作成
app = Flask(__name__)
```

```python
# 最適化問題を解く API 用の関数
@app.route('api/', methods=['POST'])
def solve():
    # 1. リクエスト受信
    students_df, cars_df = preprocess(request)
    # 2. 最適化実行
    solution_df = CarGroupProblem(students_df, cars_df).solve()
    # 3. レスポンス返送
    response = postprocess(solution_df)
    return response
```

少しコードを付け足すだけで Flask を実行できちゃう！お手軽！


```python
Flask(__name__)
```
とは何？
```python
@app.route('api/', methods=['POST'])
```
って？

- `Flask(__name__)` は Flask のクラスのインスタンスを作成する。   
    - このインスタンスが API や Web アプリケーションを管理するさまざまな役割を持つ

- `@...` は デコレータ
    - 関数の直前に書くことで、関数を修飾してくれるという特殊な関数。
    ```python
    def deco(func):
    def wrapper(*args, **kwargs):
        print('--start--')
        func(*args, **kwargs)
        print('--end--')
    return wrapper

    @deco
    def test():
        print('Hello Decorator')
    ```

- 以下が`test()`の出力  
    > --start--  
    > Hello Decorator  
    > --end--  
- [Pythonのデコレータについて](https://qiita.com/mtb_beta/items/d257519b018b8cd0cc2e)より引用

- @app.route は何をやっているの？
    - 「API サーバーの /api エンドポイント(URL) に対する、POST リクエストを受け付けた際に、@app.route で修飾する関数(solve())を実行してね」というデコレータ
    - 要するに、どこかの URL (自分で設定する末尾が'/api'の URL) で POST というリクエストを受けた時に、solve() を実行する
    - URL と solve() のように、特定の URL と関数を紐付ける機能を一般に**ルーティング**という
- POST とは？
    - 一般に情報の送信を表す。
    - 恐らく、要するに solve() へ情報を渡すためにどうにかして使われる？

リクエスト受信の部分のコード、`preprocess()` を確認しよう

```python
def preprocess(request):
    """リクエストデータを受け取り、データフレームに変換する関数"""
    # 各ファイルを取得する
    students = request.files['students']
    cars = request.files['cars']
    # pandas で読み込む
    students_df = pd.read_csv(students)
    cars_df = pd.read_csv(cars)
    
    return students_df, cars_df
    
```

- 引数 request：Flask で出力されるオブジェクト
    - POST メソッドでリクエストが投げられた時は, request の属性（request.files）にデータが格納される。

後処理部分の関数、 `postprocess()` を確認しよう

```python
def postprocess(solution_df):
    """データフレームを csv に変換する関数"""
    solution_csv = solution_df.to_csv(index=False)
    response = make_response()
    response.data = solution_csv
    response.headers['Content-Type'] = 'text/csv'
    return response
```

- このコードは最適化の結果(df)をcsvに変換し、Flask によってレスポンスとして出力している
    - `make_response()` が自動的にリクエスト元へレスポンスを返してくれている
- `make_response()` 関数で作成したオブジェクトにデータとヘッダー情報を格納
    - `response.data`, `response.headers['Content-Type']`

API 作成完了！コードを眺めよう

In [41]:
"""最適化問題を解き、最適化結果を返す API """
from flask import Flask, make_response, request
import pandas as pd

from problem import CarGroupProblem


app = Flask(__name__)

def preprocess(request):
    """リクエストデータを受け取り、データフレームに変換する関数"""
    # 各ファイルを取得する
    students = request.files['students']
    cars = request.files['cars']
    # pandas で読み込む
    students_df = pd.read_csv(students)
    cars_df = pd.read_csv(cars)
    return students_df, cars_df

def postprocess(solution_df):
    """データフレームを csv に変換する関数"""
    solution_csv = solution_df.to_csv(index=False)
    response = make_response()
    response.data = solution_df
    response.headers['Content-Type'] = 'text/csv'
    return response

@app.route('/api', methods=['POST'])
def solve():
    """最適化問題を解く API 用の関数"""
    # 1. リクエスト受信
    students_df, cars_df = preprocess(request)
    # 2. 最適化実行
    solution_df = CarGroupProblem(students_df, cars_df).solve()
    # 3. レスポンス返送
    response = postprocess(solution_df)
    return response

## 最適化 API の使用

実際に API を起動してみる。  
(Mac) ターミナルを開いて以下のコードを実行する。  
```shell
export FLASK_DEBUG=TRUE
export FLASK_APP=api:app
flask run
```
＊ FLASK_ENV コマンドは deprecated らしいので、FLASK_DEBUG に変更。

すると、ターミナル上で以下のような出力がされるはず
>  \* Serving Flask app 'api:app'  
>  \* Debug mode: on  
> WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.  
>  \* Running on http://127.0.0.1:5000  
> Press CTRL+C to quit  
>  \* Restarting with stat  
>  \* Debugger is active!  
>  \* Debugger PIN: 119-491-275  

そうしたらターミナルの別ウィンドウで、以下を入力
```shell
curl -X POST \
-F students=@resource/students.csv \
-F cars=@resource/cars.csv \
-o resource/solution.csv \
http://127.0.0.1:5000/api
```

すると以下の出力が、flask run を実行したウィンドウ上で追加されるはず
> ......  
> 127.0.0.1 - - [12/Jan/2023 17:51:12] "POST / HTTP/1.1" 200 -

日時の部分は変わるはず。一番最後の 200 は、ステータスコード。もしここが異なっていれば、solution.csv を確認してみると良いかも。

リクエストを`curl` コマンドでなく、python のコマンドで送ることも可能。

```python
import requests

# API のエンドポイント
url = 'http://127.0.0.1:5000/api'
# リクエストで渡すデータ
files = {
    'students': open('../PyOptBook-main/6.api/resource/students.csv', 'r'),
    'cars': open('../PyOptBook-main/6.api/resource/cars.csv', 'r')
}
# POST リクエスト
response = requests.post(url, files=files)
# 結果の保存（`response.text`でレスポンス内容にアクセス可能）
with open('../PyOptBook-main/6.api/resource/solution_requests.csv', 'w') as f:
    f.write(response.text)
```

# Web アプリケーションを作る

UI (特に、Web ブラウザで表示される画面)を作り、Web アプリケーションを構築しよう！  
今回の目的は綺麗な見た目のサイトを作ることではなく、フロントエンジニアにイメージを持ってもらうことなので、html のみで作成する。

【要件】  
- ブラウザから特定の URL にアクセスする  
- 学生データと車データをアップロードする
- ==> 学生の乗車グループ分けの最適化結果がダウンロードできる

【仕様】  
- HTTP プロトコルによる Web サーバーとの通信ができる
- トップページで csv ファイルをアップロードできる
- 最適化後のページで結果を表示し、csv ファイルのダウンロードができる

学生と車のデータをターミナルを使わないでアップロードできる様になるので、プログラマ以外でも利用しやすい API になる！

今回は Web ブラウザを構築するのに、HTML ファイルを扱う。

<div style="width:image width px; font-size:100%; text-align:center;"><img src='img/WebApplication_case.png' alt="alternate text" width="width" height="height" style="width:800px;height:640px;" /> Web アプリケーションのリクエストとレスポンス </div>

三つのリクエストと、三つのレスポンス

**Flask を使った Web アプリケーションの学習**  
実装するコード(application.py)の全体像を見てみる。

In [42]:
from flask import Flask, redirect, render_template, request
from problem import CarGroupProblem


app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def solve():
    # トップページを表示する（GET リクエストが来た場合）
    if request.method == 'GET':
        return render_template('index.html', solution_html=None)

    # POSTリクエストである「最適化を実行」ボタンが押された場合に実行
    # データがアップロードされているかチェックする。適切でなければ元のページ（トップページ）に戻る
    if not check_request(request):
        return redirect(request.url)

    # 前処理（データ読み込み）
    students_df, cars_df = preprocess(request)
    # 最適化実行
    solution_df = CarGroupProblem(students_df, cars_df).solve()
    # 後処理（最適化結果をHTMLに表示できる形式にする）
    solution_html = postprocess(solution_df)
    return render_template('index.html', solution_html=solution_html)

API との大きな変更点は、
1. methods= に GET が追加
2. リクエスト内容のチェック（chekc_request()関数）
3. 返り値を最適化結果の csv データではなく、render_template() 関数に変更
    - --> HTML ファイルを**レンダリング**（HTMLファイルなどに含まれるデータを解釈しどの様に画面表示するのか決める処理）する

## GET メソッドの追加

トップページを取得・表示する  
POST と GET を、if 文を使うことによって一つの関数の中にまとめているが、これを二つの関数（ルーティング）に分割することも可能

## GET リクエストが来たらトップページを表示する

solve() の直後に `if request.methods == 'GET'` を置くことで、Web アプリケーションに初めてアクセスした時にトップページを返すという処理(render_template())を行う仕様に。

## リクエスト内容のチェック

適切なデータがアップロードされているか確かめる。   
check_request(request) は、 POST リクエストが入力された、つまり「最適化を実行」ボタンが押された時に（後に実装する）、データ内容を確認するコード。   
もしファイルをアップロードせずに実行ボタンが押された際には、リクエストを投げた時点で表示していたページに戻る（**リダイレクト**）仕様にしている。

他の変更点を二つ挙げる。

## 最適化結果を HTML で表示できる形式にする

postprocess() 関数で、templates/ フォルダに作成する solution_html を出力する様にする。

## render_template() 関数

index.html を元に、web ページを表示させる

では、index.html はどの様に作るの？  
--> 詳しくは教科書を確認してみてください

```html
<!DOCTYPE html>
<!-- 1. ファイルを選択、最適化を実行 -->
<title>学生の乗車グループ分け問題</title>
<h1>学生の乗車グループ分け問題</h1>
<form action="/" method=post enctype=multipart/form-data>
    <p>学生データ<input type=file name=students></p>
    <p>車データ<input type=file name=cars></p>
    <input type=submit value=最適化を実行>
</form>

<!-- 2. 最適化結果をダウンロード、最適化結果を表示 -->
{% if solution_html %}
    <h1>最適化結果</h1>
    <!-- 最適化結果をcsv形式でダウンロード -->
    <form name=download action="/download" method=post enctype=multipart/form-data>
        <input type=hidden name=solution_html value="{{ solution_html }}">
        <p><input type=submit value=ダウンロード></p>
    </form>  

    <!-- 最適化結果を表示 -->
    {{ solution_html|safe }}
{% endif %}
```

## check_request()

index.html と Flask による Web アプリケーションのコードを結びつけるための関数を実装する

In [43]:
def check_request(request):
    """リクエストに学生データと車データが含まれているか確認する関数"""
    # 各ファイルを取得する
    students = request.files['students']
    cars = request.files['cars']

    # ファイルが選択されているか確認
    if students.filename == '':
        # 学生データが選ばれていません
        return False
    if cars.filename == '':
        # 車データが選ばれていません
        return False

    return True

## リクエストデータを受け取り、df に

In [44]:
def preprocess(request):
    """リクエストデータを受け取り、データフレームに変換する関数"""
    # 各ファイルを取得する
    students = request.files['students']
    cars = request.files['cars']
    # pandas で読み込む
    students_df = pd.read_csv(students)
    cars_df = pd.read_csv(cars)

    return students_df, cars_df

## 最適化結果を HTML 形式に

In [45]:
def postprocess(solution_df):
    """最適化結果をHTML形式に変換する関数"""
    solution_html = solution_df.to_html(header=True, index=False)
    return solution_html

## 最適化の実行と結果の表示を行う関数

```python
@app.route('/', methods=['GET', 'POST'])
def solve():
    """最適化の実行と結果の表示を行う関数"""
    # トップページを表示する（GETリクエストがきた場合）
    if request.method == 'GET':
        return render_template('index.html', solution_html=None)

    # POSTリクエストである「最適化を実行」ボタンが押された場合に実行
    # データがアップロードされているかチェックする。適切でなければ元のページ（トップページ）に戻る
    if not check_request(request):
        return redirect(request.url)

    # 前処理（データ読み込み）
    students_df, cars_df = preprocess(request)
    # 最適化実行
    solution_df = CarGroupProblem(students_df, cars_df).solve()
    # 後処理（最適化結果をHTMLに表示できる形式にする）
    solution_html = postprocess(solution_df)
    return render_template('index.html', solution_html=solution_html)
```

## 最適化表示結果を csv に変換してダウンロードする関数

```python
@app.route('/download', methods=['POST'])
def download():
    """リクエストに含まれるHTMLの表形式データをcsv形式に変換してダウンロードする関数"""
    solution_html = request.form.get('solution_html')
    solution_df = pd.read_html(solution_html)[0]
    solution_csv = solution_df.to_csv(index=False)
    response = make_response()
    response.data = solution_csv
    response.headers['Content-Type'] = 'text/csv'
    response.headers['Content-Disposition'] = 'attachment; filename=solution.csv'
    return response
```

## まとめると

```python
"""最適化を解く Webアプリケーション"""
from flask import Flask, make_response, redirect, render_template, request
import pandas as pd
from problem import CarGroupProblem


app = Flask(__name__)


def check_request(request):
    """リクエストに学生データと車データが含まれているか確認する関数"""
    # 各ファイルを取得する
    students = request.files['students']
    cars = request.files['cars']

    # ファイルが選択されているか確認
    if students.filename == '':
        # 学生データが選ばれていません
        return False
    if cars.filename == '':
        # 車データが選ばれていません
        return False

    return True


def preprocess(request):
    """リクエストデータを受け取り、データフレームに変換する関数"""
    # 各ファイルを取得する
    students = request.files['students']
    cars = request.files['cars']
    # pandas で読み込む
    students_df = pd.read_csv(students)
    cars_df = pd.read_csv(cars)

    return students_df, cars_df


def postprocess(solution_df):
    """最適化結果をHTML形式に変換する関数"""
    solution_html = solution_df.to_html(header=True, index=False)
    return solution_html


@app.route('/', methods=['GET', 'POST'])
def solve():
    """最適化の実行と結果の表示を行う関数"""
    # トップページを表示する（GETリクエストがきた場合）
    if request.method == 'GET':
        return render_template('index.html', solution_html=None)

    # POSTリクエストである「最適化を実行」ボタンが押された場合に実行
    # データがアップロードされているかチェックする。適切でなければ元のページ（トップページ）に戻る
    if not check_request(request):
        return redirect(request.url)

    # 前処理（データ読み込み）
    students_df, cars_df = preprocess(request)
    # 最適化実行
    solution_df = CarGroupProblem(students_df, cars_df).solve()
    # 後処理（最適化結果をHTMLに表示できる形式にする）
    solution_html = postprocess(solution_df)
    return render_template('index.html', solution_html=solution_html)


@app.route('/download', methods=['POST'])
def download():
    """リクエストに含まれるHTMLの表形式データをcsv形式に変換してダウンロードする関数"""
    solution_html = request.form.get('solution_html')
    solution_df = pd.read_html(solution_html)[0]
    solution_csv = solution_df.to_csv(index=False)
    response = make_response()
    response.data = solution_csv
    response.headers['Content-Type'] = 'text/csv'
    response.headers['Content-Disposition'] = 'attachment; filename=solution.csv'
    return response
```

## Web アプリケーションを起動する

ターミナル上で以下コマンドを実行する

```shell
export FLASK_DEBUG=TRUE
export FLASK_APP=application:app
flask run
```

 \* Running on http://127.0.0.1:5000  
 と表示されるはずなので、Running on 以下のアドレスに Web ブラウザから飛んでみる。  
 --> 完成！